In [1]:
#from http://www.mikaelbrunila.fi/2017/03/27/scraping-extracting-mapping-geodata-twitter/
# https://marcobonzanini.com/2015/03/02/mining-twitter-data-with-python-part-1/

import tweepy
from tweepy import OAuthHandler
import json

consumer_key = '93WP2pKvICGqCvrdA6FWMbFLB'
consumer_secret = 'LWcPKp3oqEmYTTbVaNpRD5tgqLl2KAfe2bNGsBk5xwTjxLxqHH'
access_token = '3080541417-QQ15kHSfkZYBWWSqLHwKwfqB3INeYgxpY3P9IbO'
access_secret = '2nuMVxvX8TzMyl1xHkUvElNy6BqWjqdoolTvLLxwoXHi1'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [2]:
places = api.geo_search(query="USA", granularity="country")
place_id = places[0].id

In [3]:
tweets = api.search(q="place:%s" % place_id)
for tweet in tweets:
    print(tweet.text + " | " + tweet.place.name if tweet.place else "Undefined place")

❤🐶woww wowww wowwww
🐶❤ this diving dog can https://t.co/40MG6yJ1po | Ballenger Creek
Go check out my mans youtube channel for the reviews on shoe apparel and more‼️‼️‼️ https://t.co/FZa0rPDbKO | North Carolina
https://t.co/uMVOMyncfT | Chicago
https://t.co/IBKoxW14qk | West Virginia
I See You Tank... | New Jersey
This O line is awful | Georgia
@FatyCepeda We love a supportive queen https://t.co/piK8cgNfEH | Dallas
@AndyHemmings66 Blankets are hope's nirvana https://t.co/IiIpBfmAmL | West Melbourne
But anyway I love my man with all my heart. So 10/10 I’m keeping him ❤️ | Fontana


In [4]:
def barbieri_clean_up(tweet):
    #casefold() lowercases all words. Replace corrects for emojis that are not separated by space to words. 
    raw = tweet.text.casefold().encode(encoding = 'unicode-escape').decode().replace('\\U',' \\U').split() 
    #remove all hyperlinks
    new_raw = []
    for item in raw:
        if item.find('http') == -1:
            new_raw.append(item)
    return new_raw

def emoji_count(clean_tweet):
    i = 0
    for item in clean_tweet:
        if item.find('\\U') > -1:
            i += 1
    return i

In [5]:
[print(tweet.text) for tweet in tweets]
print()
for tweet in tweets:
    clean_tweet = barbieri_clean_up(tweet)
    print(clean_tweet)
    print()
    print(emoji_count(clean_tweet))
    print()

❤🐶woww wowww wowwww
🐶❤ this diving dog can https://t.co/40MG6yJ1po
Go check out my mans youtube channel for the reviews on shoe apparel and more‼️‼️‼️ https://t.co/FZa0rPDbKO
https://t.co/uMVOMyncfT
https://t.co/IBKoxW14qk
I See You Tank...
This O line is awful
@FatyCepeda We love a supportive queen https://t.co/piK8cgNfEH
@AndyHemmings66 Blankets are hope's nirvana https://t.co/IiIpBfmAmL
But anyway I love my man with all my heart. So 10/10 I’m keeping him ❤️

['\\u2764', '\\U0001f436woww', 'wowww', 'wowwww\\n', '\\U0001f436\\u2764', 'this', 'diving', 'dog', 'can']

2

['go', 'check', 'out', 'my', 'mans', 'youtube', 'channel', 'for', 'the', 'reviews', 'on', 'shoe', 'apparel', 'and', 'more\\u203c\\ufe0f\\u203c\\ufe0f\\u203c\\ufe0f']

0

[]

0

[]

0

['i', 'see', 'you', 'tank...']

0

['this', 'o', 'line', 'is', 'awful']

0

['@fatycepeda', 'we', 'love', 'a', 'supportive', 'queen']

0

['@andyhemmings66', 'blankets', 'are', "hope's", 'nirvana']

0

['but', 'anyway', 'i', 'love', 'my', 